# PDF 읽고 내용 추출하기
Linkedin.pdf 하나 만들어놓고 시작한다.

summary.txt에 내용은 적어놓고.

In [ ]:
# ModuleNotFoundError: No module named 'pypdf'
# pypdf를 설치해야 한다.
%pip install pypdf
# gradio도 설치
%pip install gradio

In [ ]:
# 일단 import는 해야지.
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [ ]:
# 늘 하는 거
load_dotenv(override=True)
openai = OpenAI()

In [ ]:
reader = PdfReader("me/InseokPark.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [ ]:
print(linkedin)

In [ ]:
with open("me/aboutMe.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [ ]:
name = "Inseok Park"

In [ ]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website. \
    particularly questions related to {name}'s career, background, skills and experience. \
    Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
    You are given a summary of {name}'s background and linkedIn profile which you can use to answer questions. \
    Be professional and engaing, as if talking to a potential client or future employer who came across the website. \
    If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:{summary}\n\n## LinkedIn Profile:{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}"

In [ ]:
system_prompt

In [ ]:
def chat(message, history):
    message = [{"role":"system", "content":system_prompt}] + history + [{"role":"user", "content":message}]
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=message
    )
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

## 이제 응용해서 해본다.
1. LLM한테 질문을 평가해보라고 하고
2. 평가해서 bad가 나오면 다시 실행하라고 하고
3. 요걸 반복하게 한다.

all without Agentic framework!

In [ ]:
# Pydantic model 작성
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [ ]:
# evaluater의 system 프롬프트 (고정 string)
evaluator_system_prompt = f"You are an evaluator who deciedes whether a response for a question is acceptable. \
You are provided with a conversation between a user and an agent. Your task is to decide \
whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potencial customer or \
future employer who came accros the website. \
The Agent has been provided with context on {name} in the form of their summary and linkedIn profile. \
Here is the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with \
    whether the response is acceptable and your feedback."

In [ ]:
# evaluater의 유저 프롬프트 작성 함수
def evaluater_user_prompt(rely, history, message):
    user_prompt = f"Here is the conversation between the user and the Agent:\n{history} \n\n \
        Here is the latest message from the user:\n{message}\n\n \
        Here is the latest respnse from the Agent:\n{rely} \n\n \
        Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [ ]:
# Gemini 인스턴스를 생성하고
import os
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini = OpenAI(
    api_key=os.getenv("GEMINI_API_KEY"),
    base_url=GEMINI_BASE_URL
)

In [ ]:
# gemini야, evaluate를 해야지. 
def evaluate(rely, message, history) -> Evaluation:
    message = ([{"role":"system", "content":evaluator_system_prompt}] + 
               [{"role":"user", "content":evaluater_user_prompt(rely, history, message)}])
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=message, response_format=Evaluation)
    return response.choices[0].message.parsed

In [ ]:
# openai 답이 틀렸으면, 다시 답해야지
def rerun(reply, message, history, feedback):
    # system prompt로 답이 틀렸다고 알려야지.
    updated_sys_prompt = system_prompt + f"\n\n Your previous response is rejected.\nYou just tried to reply, \
        but the quality control rejected your reply.\n \
        Your attempted answer:\n{reply}\n\n \
        Reason for rejection:\n{feedback}\n\n"
    messages = ([{"role":"system", "content":updated_sys_prompt}]
                + history
                + [{"role":"user", "content":message}])
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    return response.choices[0].message.content

In [ ]:
# pydantic model에서 사용할 chat 작성
# message:user 질문
# history: 여태껏 대화내용
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\n everything you answer needs to be in pig latin. \
            It is madatory that you respond only and entirely in pig latin."
    else:
        system = system_prompt
    messages = ([{"role":"system", "content":system}]
                + history 
                + [{"role":"user", "content":message}])
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    reply = response.choices[0].message.content

    # evaluate시켜야지. gemini한테
    evaluation = evaluate(reply, message, history)

    #
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retry")
        print(f"reply:{reply}\nmessage:{message}\n")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)
    return reply 

In [ ]:
gr.ChatInterface(chat, type="messages").launch()